# Cycle GAN implementation by tensorflow

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from util import *
from ops import *

# model

In [ ]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.X_dim = 96*96*3
        self.z_dim = 100

    def __call__(self, inputs, name):
        with tf.variable_scope('g_' + name, reuse=self.reuse):
            inputs = tf.reshape(inputs, [-1, self.z_dim])
            fc1 = inputs
            fc2 = tf.layers.dense(fc1, 6*6*512)
            fc2 = tf.reshape(fc2, [-1, 6,6,512])
            fc2 = tf.nn.relu(fc2)
            conv1 = tf.contrib.layers.conv2d_transpose(fc2, 512, [4,4],[2,2]) # 12
            conv1 = tf.nn.relu(conv1)
            conv2 = tf.contrib.layers.conv2d_transpose(conv1, 256, [4,4],[2,2]) # 24
            conv2 = tf.nn.relu(conv2)
            conv3 = tf.contrib.layers.conv2d_transpose(conv2, 128, [4,4],[2,2]) # 48
            conv3 = tf.nn.relu(conv3)
            conv4 = tf.contrib.layers.conv2d_transpose(conv3, 3, [4,4],[2,2], activation_fn=tf.sigmoid)
            conv4 = tf.reshape(conv4, [-1, 96*96*3])
            outputs = conv4
        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return outputs

In [ ]:
class Discriminator:
    def __init__(self, batch_size=32):
        self.reuse = False
        self.X_dim = 96*96*3
        self.initializer = tf.contrib.layers.xavier_initializer()
        self.s_size = 6

    def __call__(self, inputs, name):
        def leaky_relu(x, leak=0.2, name='outputs'):
            return tf.maximum(x, x * leak, name=name)

        with tf.variable_scope('d_'+name, reuse=self.reuse):
            x = tf.reshape(inputs, [-1, 96, 96, 3])
            conv1 = tf.layers.conv2d(x, 64, [4,4], [2,2])
            conv1 = leaky_relu(conv1)
            conv2 = tf.layers.conv2d(conv1, 128, [4,4], [2,2])
            conv2 = leaky_relu(conv2)
            conv3 = tf.layers.conv2d(conv2, 256, [4,4], [2,2])
            conv3 = leaky_relu(conv3)
            conv4 = tf.layers.conv2d(conv3, 512, [4,4], [2,2])
            conv4 = leaky_relu(conv4)
            conv4 = tf.contrib.layers.flatten(conv4)
            fc1 = tf.layers.dense(conv4, 512)
            fc1 = leaky_relu(fc1)
            fc2 = tf.layers.dense(fc1, 256)

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return fc2

In [ ]:
# f(x) = ||h(x) - h(x_)|| - ||h(x)||
class Critic(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) - self.h(x_), axis=1) - tf.norm(self.h(x), axis=1)

# f(x) = ||h(x) - h(x_)||
class calc_Norm(object):
    def __init__(self, h):
        self.h = h
    def __call__(self, x, x_):
        return tf.norm(self.h(x) -self.h(x_))

In [ ]:
def l2_loss(x1, x2, weights=1.0):
    loss = tf.reduce_mean((x1 - x2) ** 2) * weights
    return loss

def l1_loss(x1, x2, weights=1.0):
    loss = tf.reduce_mean(tf.abs(x1 - x2)) * weights
    return loss

In [ ]:
class cycleGAN:
    def __init__(self):
        # definition of G and D for each X2Y, Y2X
        self.g_X2Y = Generator(name="X2Y")
        self.g_Y2X = Generator(name="Y2X")
        self.d_X2Y = Discriminator(name="X2Y")
        self.d_Y2X = Discriminator(name="Y2X")
        
        self.z_dim = 100
        self.critic = Critic(self.d)
        self.calcNorm = calc_Norm(self.d)

        self.X_train = np.load("irasutoya_face_1813x96x96x3_jpg.npy")
        self.X_train = self.X_train/255
        self.X_dim = 96*96*3
        self.batch_size = 32
        self.epochs = 500000
        self.display_epoch = 100
        self.param_save_epoch = 10000
        self.loss = {"d_loss_X":[], "d_loss_Y":[], "g_loss":[]}
        
        self.crop_size = 256

        self.X_tr = tf.placeholder(tf.float32, shape=[None, sefl.crop_size, self.crop_size, 3])
        self.Y_tr = tf.placeholder(tf.float32, shape=[None, self.crop_size, self.crop_size, 3])

        self.lr = 0.0002

        
    def loss_(self):
        # the results of generation
        X2Y = self.g_X2Y(X_tr) #X→Y
        Y2X = self.g_Y2X(Y_tr) #Y→X
        X2Y2X = self.g_Y2X(X2Y) #X→Y→X
        Y2X2Y = self.g_X2Y(Y2X) #Y→X→Y

        # the results of discrimination
        X_dis = self.d_Y2X(self.X_tr)
        Y2X_dis = self.d_Y2X(Y2X)
        Y_dis = self.d_X2Y(self.Y_tr)
        X2Y_dis = self.d_X2Y(X2Y)

        # g_losses
        g_loss_X2Y = tf.identity(l2_loss(X2Y_dis, tf.ones_like(X2Y_dis)))
        g_loss_Y2X = tf.identity(l2_loss(Y2X_dis, tf.ones_like(Y2X_dis)))
        cyc_loss_X = tf.identity(l1_loss(self.X_tr, X2Y2X) * 10.0)
        cyc_loss_Y = tf.identity(l1_loss(self.Y_tr, Y2X2Y) * 10.0)
        g_loss = g_loss_X2Y + g_loss_Y2X + cyc_loss_X + cyc_loss_Y

        #  d_losses
        d_loss_Xtr = l2_loss(X_dis, tf.ones_like(X_dis))
        d_loss_Y2X = l2_loss(Y2X_dis, tf.ones_like(Y2X_dis))
        d_loss_X = d_loss_Xtr + d_loss_Y2X
        
        d_loss_Ytr = l2_loss(Y_dis, tf.ones_like(Y_dis))
        d_loss_X2Y = l2_loss(X2Y_dis, tf.ones_like(X2Y_dis))
        d_loss_Y = d_loss_Ytr + d_loss_X2Y
 
        return g_loss, d_loss_X, d_loss_Y

    def train():
        g_loss, d_loss_X, d_loss_Y = self.loss()

        # Optimizer
        g_var = [var for var in (self.g_X2Y or self.g_Y2X)]
        d_X_train_op = tf.train.AdamOptimizer(self.lr, beta1=0.5).minimize(d_loss_X, var_list=self.d_Y2X)
        d_Y_train_op = tf.train.AdamOptimizer(self.lr, beta1=0.5).minimize(d_loss_Y, var_list=self.d_X2Y)
        g_train_op = tf.train.AdamOptimizer(self.lr, beta1=0.5).minimize(g_loss, var_list=g_var)

        saver = tf.train.Saver()
        #%debug
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for epoch in range(epochs):

                # X_mb, Y_mbを収集
                
                
                # train G
                _,  = sess.run([g_train_op, g_loss], feed_dict={self.X_tr: X_mb, self.Y_tr: Y_mb})
                    
                # train D_X
                _ = sess.run([d_X_train_op, d_loss_X], feed_dict={self.X_tr: X_mb, self.Y_tr: Y_mb})
                
                # train D_Y
                _ = sess.run([d_Y_train_op, d_loss_Y], feed_dict={self.X_tr: X_mb, self.Y_tr:Y_mb})
              
                # 結果をappend
                loss["d_loss_X"].append(d_loss)
                loss["d_loss_Y"].append(d_loss_value)
                loss["n_loss"].append(g_loss_value)
                print("epoch:" + str(epoch))
                # グラフの描画（余裕があったら）
                if epoch % display_epoch == 0:
                    save_metrics(loss, epoch)

                if epoch % param_save_epoch == 0:
                    path = "model"
                    if not os.path.isdir(path):
                        os.makedirs(path)

                    saver.save(sess, "./model/dcgan_model" + str(epoch) + ".ckpt")


        
    
    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        return images

# main func